# MVD 11. cvičení


## 1. část - Collaborative Filtering (item-item)

Vytvořte si matici podle příkladu v přednášce na item-item přístup na snímku 24. Využijte centered cosine similarity pro výpočet podobností stejně jako v přednášce a vyberte dva nejbližší sousedy. Ověřte, že je výsledek v přednášce správný a implementujte funkci tak, aby bylo možné jednoduše spočítat i libovolné další vyhodnocení. 

In [4]:
import numpy as np
from scipy import spatial

mat = np.array([[1,0,3,0,0,5,0,0,5,0,4,0],[0,0,5,4,0,0,4,0,0,2,1,3],[2,4,0,1,2,0,3,0,4,3,5,0], [0,2,4,0,5,0,0,4,0,0,2,0], [0,0,4,3,4,2,0,0,0,0,2,5], [1,0,3,0,3,0,0,2,0,0,4,0]], dtype=float)
idx = (0, 4)

def cossim_centred(matrix):
    matr_norm = np.copy(matrix)
    for i, row in enumerate(matr_norm):
        mean = np.mean(row)
        count = np.where(row != 0)[0].shape[0]
        suma = np.sum(row)
        mean = suma/count
        for j, item in enumerate(row):
            if item != 0:
                matr_norm[i, j] = matr_norm[i, j] - mean
    return matr_norm
    
def collab_filtr(mat, idx):
    matrix_norm = cossim_centred(mat)
    similarities = []
    for index in set(np.where(mat != mat[idx[0]])[0]):
        sim = 1 - spatial.distance.cosine(matrix_norm[idx[0]], matrix_norm[index])
        similarities.append(sim)
    near_neigh_idx = np.array(similarities).argsort()[-2:]
    sum_s_r = 0
    sum_s = 0
    for i in near_neigh_idx[::-1]:
        sum_s_r += similarities[i]*mat[i+1, idx[1]]
        sum_s += similarities[i]
    r = sum_s_r/sum_s
    return r
    
collab_filtr(mat, idx)

2.586406866934817

## Bonus - Content-based Filtering

Stáhněte si Kaggle dataset [Spotify Recommendation system](https://www.kaggle.com/bricevergnou/spotify-recommendation). Y datasetu budete pot5ebovat:

- data.csv = příznaky k jednotlivým skladbám + příznak liked pro klasifikaci

Úkolem je:

1. Načíst data z csv.
2. Vytvořit train (90 %) a test (10 %) split pomocí knihovny [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
3. Vytvořit model logistické regrese (vlastní implementace / využití knihovny sklearn nebo jiné).
4. Vyhodnotit data na testovací sadě (např. metoda score u LogisticRegression).

**Skóre pro uznání úlohy musí být vyšší než 89 %.**

Dobrovolné: vytvořit graf predikovaných vs aktuálních hodnot
Získání bodu navíc: Využijte i další algoritmy pro binární klasifikaci.

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

model = LogisticRegression(C=500)
data = pd.read_csv('data.csv')
x_train, x_test, y_train, y_test = train_test_split(data.drop('liked', axis=1), data.liked, test_size=0.1)

In [15]:
min_max_scaler = MinMaxScaler()
x_train_minmax = min_max_scaler.fit_transform(x_train)
x_test_minmax = min_max_scaler.transform(x_test)
model.fit(x_train_minmax, y_train)
predictions = model.predict(x_test_minmax)
score = model.score(x_test_minmax, y_test)
print("Score", score)

Score 0.9


C:\Users\kanyk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
